In [1]:
import re
import string
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
from nltk.tokenize import TweetTokenizer
import os
# os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding, GRU, Input, Bidirectional
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.utils import np_utils
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score


Using TensorFlow backend.
/home/vaibhav/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vaibhav/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vaibhav/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vaibhav/.local/lib/python3.6/site-packages/tensorflow/py

In [28]:
def clean_text(text):
#     print(text)
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)
    text = " ".join([w for w in twtk.tokenize(text) if w != "" and w is not None])
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [29]:
# functions for cleaning
def removeStopwords(tokens):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tokens if not tok in stops and len(tok) >= 3]
    return toks

def removeURL(text):
    newText = re.sub('http\\S+', '', text, flags=re.MULTILINE)
    return newText

def removeNum(text):
    newText = re.sub('\\d+', '', text)
    return newText

def removeHashtags(tokens):
    toks = [ tok for tok in tokens if tok[0] != '#']
#     if segment == True:
#         segTool = Analyzer('en')
#         for i, tag in enumerate(self.hashtags):
#             text = tag.lstrip('#')
#             segmented = segTool.segment(text)

    return toks

def stemTweet(tokens):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tokens]
    return stemmed_words

def evaluate(target, predicted):
    f1 = f1_score(target, predicted, average='weighted')
    acc = accuracy_score(target, predicted)
    rec = recall_score(target, predicted, average = 'macro')
    print("F1 score:   ", f1)
    print("Avg Recall: ", rec)    
    print("Accuracy:   ", acc)


In [4]:
def processTweet(tweet, remove_swords = True, remove_url = True, remove_hashtags = True, remove_num = True, stem_tweet = True):
#     text = tweet.translate(string.punctuation)   -> to figure out what it does ?
    """
        Tokenize the tweet text using TweetTokenizer.
        set strip_handles = True to Twitter username handles.
        set reduce_len = True to replace repeated character sequences of length 3 or greater with sequences of length 3.
    """
    if remove_url:
        tweet = removeURL(tweet)
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]
    if remove_hashtags:
        tokens = removeHashtags(tokens)
    if remove_swords:
        tokens = removeStopwords(tokens)
    if stem_tweet:
        tokens = stemTweet(tokens)
    text = " ".join(tokens)
    return text


In [5]:
train_data = pd.read_csv('../data/train_en.tsv',delimiter='\t',encoding='utf-8')
cleaned = train_data['text'][:5].map(lambda x: processTweet(x))
# for i,t in enumerate(cleaned):
#     print (i,train_data['text'][i]
#     print 
#     print t
#     print 

In [6]:
# tweets = train_data['text']
maxlen = 140
train_data['text'] = train_data['text'].map(lambda x: processTweet(x))
vocabulary_size = 30000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_data['text'])
X_train = tokenizer.texts_to_sequences(train_data['text'])
# print(sequences)
X_train = pad_sequences(X_train, maxlen=maxlen)
labels = train_data['HS']
Y_train = np_utils.to_categorical(labels, len(set(labels)))
V = len(tokenizer.word_index) + 1


l2_coef = 0.001
tweet = Input(shape=(maxlen,), dtype='int32')
x = Embedding(V, 128, input_length=maxlen, W_regularizer=l2(l=l2_coef))(tweet)
x = Bidirectional(layer=GRU(128, return_sequences=False, 
                            W_regularizer=l2(l=l2_coef),
                            b_regularizer=l2(l=l2_coef),
                            U_regularizer=l2(l=l2_coef)),
                  merge_mode='sum')(x)
x = Dense(len(set(labels)), W_regularizer=l2(l=l2_coef), activation="softmax")(x)

model = Model(input=tweet, output=x)

model.compile(loss='categorical_crossentropy',
                  optimizer='RMSprop',
                  metrics=['accuracy'])


model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.1)

/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(10687, 128, input_length=140, embeddings_regularizer=<keras.reg...)`
/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(128, return_sequences=False, kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg..., recurrent_regularizer=<keras.reg...)`


/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="softmax", kernel_regularizer=<keras.reg...)`
/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8100 samples, validate on 900 samples
Epoch 1/10
8100/8100 [==============================] - 55s 7ms/step - loss: 0.7361 - acc: 0.7107 - val_loss: 1.2540 - val_acc: 0.4267
Epoch 2/10
8100/8100 [==============================] - 49s 6ms/step - loss: 0.5554 - acc: 0.7632 - val_loss: 0.8940 - val_acc: 0.5856
Epoch 3/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.5438 - acc: 0.7681 - val_loss: 0.9117 - val_acc: 0.5867
Epoch 4/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.5351 - acc: 0.7728 - val_loss: 0.8409 - val_acc: 0.5867
Epoch 5/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.5268 - acc: 0.7730 - val_loss: 1.1069 - val_acc: 0.5467
Epoch 6/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.5222 - acc: 0.7788 - val_loss: 0.8753 - val_acc: 0.5911
Epoch 7/10
8100/8100 [==========================

In [8]:
test_data = pd.read_csv('../data/dev_en.tsv',delimiter='\t',encoding='utf-8')
cleaned_test = test_data['text'][:5].map(lambda x: processTweet(x))

In [9]:
maxlen = 140
test_data['text'] = test_data['text'].map(lambda x: processTweet(x))
vocabulary_size = 30000
# tokenizer = Tokenizer(num_words= vocabulary_size)
# tokenizer.fit_on_texts(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_data['text'])
# print(sequences)
X_test = pad_sequences(X_test, maxlen=maxlen)
labels = test_data['HS']
Y_test = np_utils.to_categorical(labels, len(set(labels)))

In [25]:
y_predict = model.predict(X_test)
print(y_predict)
y_predicted = []
for i in y_predict:
    if i[0] < i[1]:
        y_predicted.append(1)
    else:
        y_predicted.append(0)
        
y_test = []
for i in Y_test:
    if i[0] == 1:
        y_test.append(0)
    else:
        y_test.append(1)

[[0.6518227  0.3481773 ]
 [0.97245544 0.02754457]
 [0.23733237 0.76266766]
 ...
 [0.00919168 0.9908083 ]
 [0.02382755 0.97617245]
 [0.7291961  0.27080396]]


In [26]:
evaluate(y_test, y_predicted)
# print(y_predicted[0][0])

F1 score:    0.6476282377471362
Avg Recall:  0.6370064290414474
Accuracy:    0.65
